In [10]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# device configuration.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# define ultra-parameters.
sequence_length = 28
input_size = 28
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 100
num_epochs = 5
learning_rate = 0.003

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset =  torchvision.datasets.MNIST(root='./data',
                                           train=False,
                                           transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


# Bidirectional recurrent neural network (many-to-one)
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size=input_size,
                            hidden_size=hidden_size,
                            num_layers=num_layers,
                            batch_first=True,
                            bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x, h0, c0):
        # set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

        # forward propagate LSTM
        out, states = self.lstm(x, (h0, c0))

        # decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out, states


model = BiRNN(input_size, hidden_size, num_layers, num_classes).to(device)
print(model)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# train the model
h, c = None, None
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)

        # forward pass
        outputs, states = model(images, h, c)
        h, c = states
        loss = criterion(outputs, labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch: [{}/{}], step: [{}/{}], loss: {:.5f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs, _ = model(images, h, c)
        _, prediction = torch.max(outputs, dim=1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

BiRNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=10, bias=True)
)
Epoch: [1/5], step: [100/600], loss: 0.53352
Epoch: [1/5], step: [200/600], loss: 0.14682
Epoch: [1/5], step: [300/600], loss: 0.46576
Epoch: [1/5], step: [400/600], loss: 0.15832
Epoch: [1/5], step: [500/600], loss: 0.17297
Epoch: [1/5], step: [600/600], loss: 0.15293
Epoch: [2/5], step: [100/600], loss: 0.11419
Epoch: [2/5], step: [200/600], loss: 0.12407
Epoch: [2/5], step: [300/600], loss: 0.03501
Epoch: [2/5], step: [400/600], loss: 0.02681
Epoch: [2/5], step: [500/600], loss: 0.09127
Epoch: [2/5], step: [600/600], loss: 0.06598
Epoch: [3/5], step: [100/600], loss: 0.01248
Epoch: [3/5], step: [200/600], loss: 0.01140
Epoch: [3/5], step: [300/600], loss: 0.04212
Epoch: [3/5], step: [400/600], loss: 0.07130
Epoch: [3/5], step: [500/600], loss: 0.06432
Epoch: [3/5], step: [600/600], loss: 0.01871
Epoch: [4/5], step: [100/600], loss: 0.12380
